In [8]:
import urllib.request
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import re
from datetime import datetime
import numpy as np
from configparser import ConfigParser
import unidecode
import requests
import uuid

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

# get all lidl offer url

In [9]:
def get_all_link_lidl():
    
    offer_page = 'https://www.lidl.hu/ajanlataink'
    
    page = urllib.request.urlopen(offer_page)
    soup = bs(page)
    
    divs_body = soup.body.findAll('div', {'class' : ['tabnavaccordion__content']})

    all_link = []
    for div in divs_body:
        for a in div.find_all('a', href=True):
            url = 'https://www.lidl.hu' + a['href']
            #print(url)
            all_link.append(url)
            
    return all_link

In [10]:
all_link = get_all_link_lidl()

In [11]:
all_link

['https://www.lidl.hu/c/haztartasi-es-furdoszobai-ajanlataink/c3595/w1',
 'https://www.lidl.hu/c/barkacs-es-kerti-gep-ajanlataink/c3592/w1',
 'https://www.lidl.hu/c/akcioink-08-11-csutortoktol/c3597/w1',
 'https://www.lidl.hu/c/zoldseg-gyumolcs-akcioink-08-11-csutortoktol/c3598/w1',
 'https://www.lidl.hu/c/friss-halkinalatunk/c3599/w1',
 'https://www.lidl.hu/c/virag-ajanlataink-08-11-csutortoktol/c3600/w1',
 'https://www.lidl.hu/c/sor-es-whiskey-ajanlataink/c3601/w1',
 'https://www.lidl.hu/c/jegkrem-ajanlataink/c3602/w1',
 'https://www.lidl.hu/c/haztartasi-ajanlataink/c3603/w1',
 'https://www.lidl.hu/c/furdoszobai-ajanlataink/c3604/w1',
 'https://www.lidl.hu/c/irodaszerek-es-gyermekruhazat/c3605/w1',
 'https://www.lidl.hu/c/szuper-hetvege-08-13-szombattol/c3606/w1',
 'https://www.lidl.hu/c/akcioink-08-11-csutortoktol/c3597/w2',
 'https://www.lidl.hu/c/sor-es-whiskey-ajanlataink/c3601/w2',
 'https://www.lidl.hu/c/jegkrem-ajanlataink/c3602/w2',
 'https://www.lidl.hu/c/konyhai-ajanlataink

In [19]:
def image_download(img_url):
    
    uuid_image = uuid.uuid1()
    res = requests.get(img_url)

    if res.headers['Content-Type'][res.headers['Content-Type'].find('/')+1:] == 'jpeg':

        image_name = str(uuid_image) + '.jpg'

    if res.headers['Content-Type'][res.headers['Content-Type'].find('/')+1:] == 'plain':

        image_name = str(uuid_image) + '.png'

    img_path = '/Users/kuligabor/Documents/HotDealsHungary/img/' + image_name
    urllib.request.urlretrieve(img_url , img_path)
    
    return image_name

In [20]:
def get_all_offer_lidl(all_link):
    
    def find_nth_occurrence(string, char, occurrence):

        val = -1
        for i in range(0, occurrence):
            val = string.find(char, val + 1)
        return val

    all_items = []
    
    counter = 1

    for url in all_link:

        print(f'crawl url: {url} done {counter} from {len(all_link)}')
        counter = counter + 1

        page = urllib.request.urlopen(url)
        soup = bs(page)

        divs = soup.body.findAll('div', {'class' : ['nuc-a-flex-item']})

        for div in divs:
            articles = div.findAll('article', {'class' : 'ret-o-card'})
            for article in articles:

                item_dict = {}

                #print(article['data-id'])
                item_dict['itemId'] = article['data-id']

                #print(article['data-name'])
                brand = article.find('p', {'class' : 'ret-o-card__content'})

                if brand != None:
                    #print(brand.get_text().strip())
                    item_dict['itemName'] = article['data-name'] + ' - ' + brand.get_text().strip()
                else:
                    item_dict['itemName'] = article['data-name']
                    
                item_dict['itemCleanName'] = unidecode.unidecode(item_dict['itemName']).lower()
                
                images = article.findAll('img')
                img_url = np.nan
                for img in images:
                    img_url = img['src']
                    break
                
                item_dict['imageUrl'] = image_download(img_url)
                
                #print(article['data-price'])
                item_dict['price'] = article['data-price']
                
                measure = article.find('div', {'class' : 'lidl-m-pricebox__basic-quantity'})
                
                if measure != None:
                    item_dict['measure'] = measure.get_text()
                else:
                    item_dict['measure'] = np.nan

                #print(article['data-list'])

                sales_from_pattern = r'(?P<group_1>[\d]{2}.[\d]{2})'

                if (article['data-list'] != None) and (re.search(sales_from_pattern, article['data-list']) != None):

                    sales_data = article['data-list']
                    item_dict['salesStart'] = str(datetime.now().year) + '.' + re.search(sales_from_pattern,
                                                                                          article['data-list'])[0]
                else:
                    item_dict['salesStart'] = np.nan

                cut_url = url[find_nth_occurrence(url, '/', 4)+1:]

                item_dict['source'] = cut_url[:cut_url.find('/')]
                item_dict['runDate'] = datetime.now().strftime('%Y.%m.%d-%H:%M:%S')
                item_dict['shopName'] = 'lidl'
                item_dict['isSales'] = 1

                if len(item_dict) > 0:
                    all_items.append(item_dict)

                #print('-----')

    df = pd.DataFrame(all_items)
    
    return df

In [21]:
df = get_all_offer_lidl(all_link)

crawl url: https://www.lidl.hu/c/haztartasi-es-furdoszobai-ajanlataink/c3595/w1 done 1 from 20
crawl url: https://www.lidl.hu/c/barkacs-es-kerti-gep-ajanlataink/c3592/w1 done 2 from 20
crawl url: https://www.lidl.hu/c/akcioink-08-11-csutortoktol/c3597/w1 done 3 from 20
crawl url: https://www.lidl.hu/c/zoldseg-gyumolcs-akcioink-08-11-csutortoktol/c3598/w1 done 4 from 20
crawl url: https://www.lidl.hu/c/friss-halkinalatunk/c3599/w1 done 5 from 20
crawl url: https://www.lidl.hu/c/virag-ajanlataink-08-11-csutortoktol/c3600/w1 done 6 from 20
crawl url: https://www.lidl.hu/c/sor-es-whiskey-ajanlataink/c3601/w1 done 7 from 20
crawl url: https://www.lidl.hu/c/jegkrem-ajanlataink/c3602/w1 done 8 from 20
crawl url: https://www.lidl.hu/c/haztartasi-ajanlataink/c3603/w1 done 9 from 20
crawl url: https://www.lidl.hu/c/furdoszobai-ajanlataink/c3604/w1 done 10 from 20
crawl url: https://www.lidl.hu/c/irodaszerek-es-gyermekruhazat/c3605/w1 done 11 from 20
crawl url: https://www.lidl.hu/c/szuper-hetveg

In [18]:
df

,itemId,itemName,itemCleanName,imageUrl,price,measure,salesStart,source,runDate,shopName,isSales
0,346449,Akkus haj- és szakállvágó - SILVERCREST PERSON...,akkus haj- es szakallvago - silvercrest person...,/Users/kuligabor/Documents/HotDealsHungary/img...,4444,/db,NaN,haztartasi-es-furdoszobai-ajanlataink,2022.08.12-20:42:17,lidl,1
1,373304,Kozmetikai táska / manikűrkészlet - LIVARNO HOME,kozmetikai taska / manikurkeszlet - livarno home,/Users/kuligabor/Documents/HotDealsHungary/img...,2299,/db; /készlet,NaN,haztartasi-es-furdoszobai-ajanlataink,2022.08.12-20:42:18,lidl,1
2,376140,Haj- / szakállvágó eszközök - LIVARNO HOME,haj- / szakallvago eszkozok - livarno home,/Users/kuligabor/Documents/HotDealsHungary/img...,1999,/db,NaN,haztartasi-es-furdoszobai-ajanlataink,2022.08.12-20:42:18,lidl,1
3,373306,Szakállápoló eszközök / manikűrkészlet - LIVAR...,szakallapolo eszkozok / manikurkeszlet - livar...,/Users/kuligabor/Documents/HotDealsHungary/img...,1299,/db; /készlet,NaN,haztartasi-es-furdoszobai-ajanlataink,2022.08.12-20:42:18,lidl,1
4,393751,Állóventilátor - SILVERCREST,alloventilator - silvercrest,/Users/kuligabor/Documents/HotDealsHungary/img...,8888,/db,NaN,haztartasi-es-furdoszobai-ajanlataink,2022.08.12-20:42:19,lidl,1
...,...,...,...,...,...,...,...,...,...,...,...
312,6402746,ESL tej - PILOS,esl tej - pilos,/Users/kuligabor/Documents/HotDealsHungary/img...,419,"1,5 l, 1 l = 280 Ft",NaN,xxl-ajanlataink,2022.08.12-20:44:42,lidl,1
313,11207,Extra szűz olívaolaj - PRIMADONNA,extra szuz olivaolaj - primadonna,/Users/kuligabor/Documents/HotDealsHungary/img...,2499,1 l,NaN,xxl-ajanlataink,2022.08.12-20:44:42,lidl,1
314,68193,Narancslé* - SOLEVITA,narancsle* - solevita,/Users/kuligabor/Documents/HotDealsHungary/img...,779,"2 l, 1 l = 390 Ft",NaN,xxl-ajanlataink,2022.08.12-20:44:43,lidl,1
315,146683,Lasagne bolognese - CHEF SELECT,lasagne bolognese - chef select,/Users/kuligabor/Documents/HotDealsHungary/img...,1899,"1,15 kg, 1 kg = 1652 Ft",NaN,xxl-ajanlataink,2022.08.12-20:44:43,lidl,1


In [7]:
df = df.fillna('N.a')

In [8]:
df.loc[~df['price'].str.isnumeric()]

,itemId,itemName,itemCleanName,imageUrl,price,measure,salesStart,source,runDate,shopName,isSales


In [9]:
df[df['imageUrl'].isnull()]

,itemId,itemName,itemCleanName,imageUrl,price,measure,salesStart,source,runDate,shopName,isSales


In [10]:
df

,itemId,itemName,itemCleanName,imageUrl,price,measure,salesStart,source,runDate,shopName,isSales
0,135314,Sorbet jégkrém - GELATELLI,sorbet jegkrem - gelatelli,https://hu.cat-ret.assets.lidl/catalog5media/h...,1099,"4 x 60 ml, 1 l = 4580 Ft",N.a,kostolja-meg-a-karibi-izvilagot,2022.07.21-08:02:30,lidl,1
1,163232,Garnéla - CARIBBEAN STYLE,garnela - caribbean style,https://hu.cat-ret.assets.lidl/catalog5media/h...,1999,"250 g, 1 kg = 7996 Ft",N.a,kostolja-meg-a-karibi-izvilagot,2022.07.21-08:02:30,lidl,1
2,164844,Kókuszos párnák* - CARIBBEAN STYLE,kokuszos parnak* - caribbean style,https://hu.cat-ret.assets.lidl/catalog5media/h...,1199,"500 g, 1 kg = 2398 Ft",N.a,kostolja-meg-a-karibi-izvilagot,2022.07.21-08:02:30,lidl,1
3,148837,Aszalt mangószeletek* - CARIBBEAN STYLE,aszalt mangoszeletek* - caribbean style,https://hu.cat-ret.assets.lidl/catalog5media/h...,999,"125 g, 1 kg = 7992 Ft",N.a,kostolja-meg-a-karibi-izvilagot,2022.07.21-08:02:30,lidl,1
4,161398,Ostya* - CARIBBEAN STYLE,ostya* - caribbean style,https://hu.cat-ret.assets.lidl/catalog5media/h...,899,"400 g, 1 kg = 2248 Ft",N.a,kostolja-meg-a-karibi-izvilagot,2022.07.21-08:02:30,lidl,1
...,...,...,...,...,...,...,...,...,...,...,...
292,6470,Pom-Bär original,pom-bar original,https://hu.cat-ret.assets.lidl/catalog5media/h...,539,"110 g, 1 kg = 4900 Ft",N.a,nagy-markak-hete,2022.07.21-08:02:39,lidl,1
293,2539,Pöttyös multipack - ALGIDA,pottyos multipack - algida,https://hu.cat-ret.assets.lidl/catalog5media/h...,1299,"6x40 / 4x90 ml, 1 l = 5413 / 3609 Ft",N.a,nagy-markak-hete,2022.07.21-08:02:39,lidl,1
294,6405847,Világos sör - KŐBÁNYAI,vilagos sor - kobanyai,https://hu.cat-ret.assets.lidl/catalog5media/h...,129,"0,33 l, 1 l = 391 Ft",N.a,nagy-markak-hete,2022.07.21-08:02:39,lidl,1
295,9391,All in One mosókapszula - ARIEL,all in one mosokapszula - ariel,https://hu.cat-ret.assets.lidl/catalog5media/h...,1899,"20 db, 1 db = 95 Ft",N.a,nagy-markak-hete,2022.07.21-08:02:39,lidl,1


In [11]:
df.to_excel('lidl_result.xlsx', index=False)

# backup

In [8]:
all_items = []

divs = soup.body.findAll('div', {'class' : ['nuc-a-flex-item']})

for div in divs:
    articles = div.findAll('article', {'class' : 'ret-o-card'})
    for article in articles:
            
            item_dict = {}
            
            #print(article['data-id'])
            item_dict['item_id'] = article['data-id']
            
            #print(article['data-name'])
            brand = article.find('p', {'class' : 'ret-o-card__content'})
            
            if brand != None:
                #print(brand.get_text().strip())
                item_dict['item_name'] = article['data-name'] + ' - ' + brand.get_text().strip()
            else:
                item_dict['item_name'] = article['data-name']
            
            #print(article['data-price'])
            item_dict['price'] = article['data-price']
            
            item_dict['measure'] = np.nan
            
            #print(article['data-list'])
            
            sales_from_pattern = r'(?P<group_1>[\d]{2}.[\d]{2})'
            
            if (article['data-list'] != None) and (re.search(sales_from_pattern, article['data-list']) != None):
            
                sales_data = article['data-list']
                item_dict['sales_start'] = str(datetime.now().year) + '.' + re.search(sales_from_pattern,
                                                                                      article['data-list'])[0]
            else:
                item_dict['sales_start'] = np.nan
                
            cut_url = url[find_nth_occurrence(url, '/', 4)+1:]
            
            item_dict['source'] = cut_url[:cut_url.find('/')]
            item_dict['run_date'] = datetime.now().strftime('%Y.%m.%d-%H:%M:%S')
            
            if len(item_dict) > 0:
                all_items.append(item_dict)
            
            #print('-----')

df = pd.DataFrame(all_items)

NameError: name 'soup' is not defined

In [ ]:
df